In [7]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
#import tensorflow_addons as tfa
import numpy as np
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os

# Removes an annoying Tensorflow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

    

In [8]:
def intList2Sentence(ids, wList):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    myStr = ""
    listOfResponses=[]
    for num in ids:
        if (num[0] == EOStokenIndex or num[0] == padTokenIndex):
            listOfResponses.append(myStr)
            myStr = ""
        else:
            myStr = myStr + wList[num[0]] + " "
    if myStr:
        listOfResponses.append(myStr)
    listOfResponses = [i for i in listOfResponses if i]
    return listOfResponses

def getTestInput(inputMessage, wList, maxLen):
    encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
    inputSplit = inputMessage.lower().split()
    for index,word in enumerate(inputSplit):
        try:
            encoderMessage[index] = wList.index(word)
        except ValueError:
            continue
    encoderMessage[index + 1] = wList.index('<EOS>')
    encoderMessage = encoderMessage[::-1]
    encoderMessageList=[]
    for num in encoderMessage:
        encoderMessageList.append([num])
    return encoderMessageList

In [12]:

# Load in hyperparamters
with open("wordList.txt","rb") as fp:
    wordList = pickle.load(fp)

wordList.append('<pad>')
wordList.append('<EOS>')
vocabSize = len(wordList)
# Load in hyperparamters
batchSize = 24
maxEncoderLength = 15
maxDecoderLength = 15
lstmUnits = 112
numLayersLSTM = 3

tf.reset_default_graph()
encoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
decoderLabels = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
decoderInputs = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
feedPrevious = tf.placeholder(tf.bool)
encoderLSTM = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True)

decoderOutputs, decoderFinalState = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoderInputs, decoderInputs, encoderLSTM,vocabSize,vocabSize,embedding_size= embeddingDim, feed_previous = feedPrevious)

decoderPrediction = tf.argmax(decoderOutputs,2)

# lossWeights = [tf.ones_like(l,dtype= tf.float32) for l in decoderLabels]
# loss = tf.contrib.legacy_seq2seq.sequence_loss(decoderOutputs,decoderLabels,lossWeights,vocabSize)
# optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess = tf.Session()
saver = tf.train.Saver()
#loading a saved model
#saver.restore(sess,tf.train.latest_checkpoint('models/'))
sess.run(tf.global_variables_initializer())

#forwarding result to tensorboard
# tf.summary.scalar('loss',loss)
# merged= tf.summary.merge_all()
# logDir = "tensorboard/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
# writer = tf.summary.FileWriter(logDir, sess.graph)
 # test strings
# encoderTestStrings = ["Hi","hey ! who are you","hi there","you wanna chill","bob n vagene"]
zeroVector = np.zeros((1),dtype= 'int32')

QString = input(">")
while(QString != "exit" or QString != "quit"):
    print("Q:",QString)
    inputVector = getTestInput(encoderTestStrings[num],wordList,maxEncoderLength)
    feedDict = {encoderInputs[t]:inputVector[t] for t in range(maxEncoderLength)}
    feedDict.update({decoderLabels[t]:zeroVector for t in range(maxDecoderLength)})
    feedDict.update({decoderInputs[t]:zeroVector for t in range(maxDecoderLength)})
    feedDict.update({feedPrevious: True})
    ids = (sess.run(decoderPrediction,feed_dict= feedDict))
    print("Shiba:",intList2Sentence(ids,wordList))
    QString = input(">")

def pred(inputString):
    inputVector = model.getTestInput(inputString, wordList, maxEncoderLength)
    feedDict= {encoderInputs[t]: inputVector[t] for t in range(maxEncoderLength)}
    feedDict.update({decoderLabels[t]: zeroVector for t in range(maxDecoderLength)})
    feedDict.update({decoderInputs[t]: zeroVector for t in range(maxDecoderLength)})
    feedDict.update({feedPrevious: True})
    ids = (sess.run(decoderPrediction, feed_dict=feedDict))
    return model.idsToSentence(ids, wordList)




>hi


NameError: name 'Qstring' is not defined